# Setup Label Studio ML Backend

In [ ]:
# Mount google drive because it contains pre trained model files
# which will be used in pre labelling

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download the custom ml backend for label studio ui

!git clone https://github.com/mrx7355608/labelstudio-ml-backend

%cd /content/labelstudio-ml-backend
!pip install -r requirements.txt
!pip install redis rq ultralytics pyngrok

In [ ]:
# Get IP address of colab instance
!ip a

In [ ]:
# Kill existing label studio ml backend running on port 9090
!fuser -k 9090/tcp
!kill -9 $(pgrep -f label-studio-ml)

# Set the env variables
import os
os.environ["LABEL_STUDIO_API_KEY"] = "LABEL STUDIO LEGACY API KEY"
os.environ["LABEL_STUDIO_URL"] = "IP OF COLAB INSTANCE"

# Start the backend
%cd /content
!nohup label-studio-ml start labelstudio-ml-backend --port 9090 > backend.log 2>&1 &

In [ ]:
# See logs to troubleshoot errors
!cat backend.log

# Setup Label Studio UI

Now we need to setup label studio ui which will be used to label the dataset. Our Pre trained model and AWS S3 storage will connect to label studio ui. Then the model will be used to run predictions on the new dataset stored in AWS S3 bucket. After that we will download the Labelled dataset from Label Studio UI in `JSON` format

In [ ]:
# First create a virtual environment "venv"
!python -m venv venv

In [ ]:
# Install Label Studio UI for labelling dataset
!source venv/bin/activate && pip install label-studio

In [ ]:
# Bypass csrf errors in label studio ui authentication by
# allowing all ngrok domains

!echo "CSRF_TRUSTED_ORIGINS=['https://*.ngrok-free.app']" >> venv/lib/python3.11/site-packages/label_studio/core/settings/label_studio.py

In [ ]:
# Start Label Studio UI

!source venv/bin/activate
!nohup venv/bin/label-studio start --port 8081 &

In [ ]:
# Expose label studio ui through ngrok because we cannot
# run it on colab IP address

from pyngrok import ngrok

ngrok.set_auth_token("NGROK AUTH TOKEN")
public_url = ngrok.connect(8081, hostname="NGROK STATIC DOMAIN")
print(public_url)

# Stop everything before leaving colab

In [ ]:
!fuser -k 8081/tcp
!pkill -f ngrok
!fuser -k 9090/tcp
!kill -9 $(pgrep -f label-studio-ml)